Librerías

In [3]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv

# Cargar las variables de entorno del archivo .env
load_dotenv(override=True)

# Leer las variables definidas en el archivo .env
endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX")
admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

# Crear el objeto de credenciales
credential = AzureKeyCredential(admin_key)


In [4]:
from azure.search.documents import SearchClient
import pandas as pd

search_client = SearchClient(endpoint, index_name, credential)

def display_results(results):
    df = pd.json_normalize(list(results)).dropna(axis=1, how='all')
    df["chunk"] = df["chunk"].apply(lambda c: c[:300] + '...' if len(c) > 300 else c)
    first_cols = ['title', 'chunk', '@search.score']
    df = df[first_cols + [col for col in df.columns if col not in first_cols]]

    df = df.style.set_properties(**{
        'max-width': '500px',
        'text-align': 'left',
        'white-space': 'normal',
        'word-wrap': 'break-word'
    }).hide(axis="index")


    return df

In [5]:
results = search_client.search(search_text="Agricultural crops are grown in a variety of regions", top=5, select=["title", "chunk"])

display_results(results)

title,chunk,@search.score
bhattarai2017.pdf,"in the atmo- sphere, a phenomenon termed the carbon fertilization effect (Heimann and Reichstein, 2008; McGrath and Lobell, 2013). It has also been predicted that the southern hemisphere may suffer more adverse consequences of climate change than the northern hemisphere due to differences in carbon ...",11.617579
bhattarai2017.pdf,"were gathered from the Soil Survey Geographic Database and data on crop rotations was derived from CropScape, a geospatial cropland data layer product of the US Na- tional Agricultural Statistics Service (NASS). Our findings indicate that 20-year mean yields of both corn and soy- bean for 2080–2099 ...",11.288901
bhattarai2017.pdf,"atmospheric CO2 and fixed atmospheric CO2 concentration of 350 ppm under RCP 2.6 catego- ry is 2%, whereas the difference is 11% and 20% for category RCP 4.5 and RCP8.5 respectively. The results also suggest that although carbon fertil- ization has positive effects on soybean yields, it cannot compe...",9.928432
pinhero2016.pdf,"the final tuber quality and consumer accept- ability. The effect of diseases and pests and sprouting has been discussed elsewhere in this chapter (see Sections 5 and 6.3). Potatoes are rich sources of carbohydrates, various minerals, vitamins, and phytochemicals. Storage conditions affect nutriti...",7.674674
pinhero2016.pdf,"be free from soil, sods, stones, and other foreign materials, for proper circulation of ventilation air. Storage either in bulk or in bins or sacks depends on the need, convenience, and labor cost. Potatoes are stored in jute or polypropylene sacks when the labor cost is low in countries such as ...",7.626579


In [6]:
from azure.search.documents.models import VectorizableTextQuery

results = search_client.search(vector_queries=[VectorizableTextQuery(text="Agricultural crops are grown in a variety of regions", k_nearest_neighbors=50, fields="text_vector")], top=5, select=["title", "chunk"])

display_results(results)

title,chunk,@search.score
bhattarai2017.pdf,"patterns. For example, differential yield impacts on C4 and C3 plants may affect crop profitability and alter farmers' crop choices. Here, we present a systematic approach to projecting agricultural yields under various climate scenarios by integrating the Environment Policy Integrated Climate (EPI...",0.862023
bhattarai2017.pdf,"were gathered from the Soil Survey Geographic Database and data on crop rotations was derived from CropScape, a geospatial cropland data layer product of the US Na- tional Agricultural Statistics Service (NASS). Our findings indicate that 20-year mean yields of both corn and soy- bean for 2080–2099 ...",0.857871
bhattarai2017.pdf,"of crops and prac- tices worldwide, to reduce the simulations to amanageable size, a set of representative crops and an appropriate study area were chosen. Corn and soybeans were selected for the study as these two crops combined cover almost 53% of the United States' total agricultural land (USDA N...",0.856298
bhattarai2017.pdf,"Agricultural Systems 152 (2017) 90–99 Contents lists available at ScienceDirect Agricultural Systems j ourna l homepage: www.e lsev ie r .com/ locate /agsy Projecting corn and soybeans yields under climate change in a Corn Belt watershed Mukesh Dev Bhattarai a,⁎, Silvia Secchi b, Justin Schoof b ...",0.855997
bhattarai2017.pdf,http://refhub.elsevier.com/S0308-521X(16)30917-9/rf0110 http://refhub.elsevier.com/S0308-521X(16)30917-9/rf0115 http://refhub.elsevier.com/S0308-521X(16)30917-9/rf0115 http://refhub.elsevier.com/S0308-521X(16)30917-9/rf0120 http://refhub.elsevier.com/S0308-521X(16)30917-9/rf0120 http://refhub.elsevi...,0.853016


In [7]:
results = search_client.search(
    search_text="Agricultural crops are grown in a variety of regions",
    vector_queries=[VectorizableTextQuery(text="Agricultural crops are grown in a variety of regions", k_nearest_neighbors=50, fields="text_vector")],
    top=5,
    select=["title", "chunk"]
)

display_results(results)

title,chunk,@search.score
bhattarai2017.pdf,"were gathered from the Soil Survey Geographic Database and data on crop rotations was derived from CropScape, a geospatial cropland data layer product of the US Na- tional Agricultural Statistics Service (NASS). Our findings indicate that 20-year mean yields of both corn and soy- bean for 2080–2099 ...",0.032787
bhattarai2017.pdf,"in the atmo- sphere, a phenomenon termed the carbon fertilization effect (Heimann and Reichstein, 2008; McGrath and Lobell, 2013). It has also been predicted that the southern hemisphere may suffer more adverse consequences of climate change than the northern hemisphere due to differences in carbon ...",0.031373
bhattarai2017.pdf,"patterns. For example, differential yield impacts on C4 and C3 plants may affect crop profitability and alter farmers' crop choices. Here, we present a systematic approach to projecting agricultural yields under various climate scenarios by integrating the Environment Policy Integrated Climate (EPI...",0.027903
bhattarai2017.pdf,"CDL data, we assumed the pattern of these rotations prac- tices to be constant for the entire simulation period. However, the rota- tion practices fluctuated over the period of 1974 to 2010. Stern et al. (2012) pointed out in their research that crop rotation practices have in- deed fluctuated, prim...",0.027651
bhattarai2017.pdf,"6 also suggests that the corn yields have a wider range than soy- bean yields at the beginning of the century. Such a wide range may be the reason why the ensemble mean yields of corn are different for the three RCPs even in the initial simulation period, when the differences in the impact of climat...",0.027584
